# Dobble Generator
## aka SpotIt
Algorithm from here: https://math.stackexchange.com/questions/1303497/what-is-the-algorithm-to-generate-the-cards-in-the-game-dobble-known-as-spo

### Print Dobble identifiers
This code will print valid image ids for Dobble card decks of different sizes.  
The number of images on a card must be a prime number + 1. 

### *** Requires emoji text file, since the one provided by Padraic has some broken emojis that don't display. I have uploaded this textfile alongside the notebook. ***

In [ ]:
# nIm - 1 must be prime
# Cards must have 3, 4, 6 or 8 images

nIm = 8
n = nIm - 1
r = range(n)
rp1 = range(n+1)
c = 0
#validcards = dict()
validcards = []

# First card
card = []
c += 1
for i in rp1:
    card += [i+1]
#validcards[c] = set(card)
validcards.append(list(card))

# n following cards
for j in r:
    card = [1,]
    c = c+1
    for k in r:
        card += [n+2 + n*j +k]
    #validcards[c] = set(card)
    validcards.append(list(card))
    
# n x n following cards
for i in r:
    for j in r:
        c = c+1
        card = [(i+2),]
        for k in r:
            card+= [((n+1 +n*k + (i*k+j) % n)+1)]
        #validcards[c] = set(card)
        validcards.append(list(card))


The checkValidity() function is not the most efficient method of checking whether any of the two cards have more than 1 value in common. I am aware it would be better to use the intersection method using set(), but since I decided to use lists, I felt that this way as more appropriate (even though I could convert the lists to a set first, and check them that way). Either way, this works (as we can say it returns true at the end), but it does take a few seconds to complete (when verbose = true, otherwise it's almost instantaneous). 

In [ ]:
def checkValidity(validcards, verbose = False):
    """ This function checks whether the cards in our validdeck have only 1 number in common with each other
        card in the deck. It takes two arguments, a deck (as a list) and verbose statement (true/false) set to false by 
        default. If verbose is false, the function executes without producing outputs and tells us if our deck is valid or 
        not. If we pass verbose = true to the function, it will display each pair so we can investigate any card issues.
        
        It assumes that the algorithm used to design the deck works correctly.
    """
    if verbose == False:
        for i in validcards:
            for j in validcards:
                trueCounter = 0
                if i == j:
                    pass # since these cards are the same index in the deck
                else:
                    for k in i:
                        for p in j:
                            if k == p:
                                trueCounter +=1
                if trueCounter > 1:
                    return False
                else:
                    continue
        return True
    
    else:
        for i in validcards:
            for j in validcards:
                print("Card 1:",i ,",","Card 2:",j)
                trueCounter = 0
                if i == j:
                    pass # since these cards are the same index in the deck
                else:
                    for k in i:
                        for p in j:
                            if k == p:
                                trueCounter +=1
                if trueCounter > 1:
                    print("Error with the deck. Match found!")
                    return False
                else:
                    print("No match between card 1 and card 2. Continuing search...")
                    continue
        return True

                    
                    
    

In [ ]:
checkValidity(validcards) # add verbose = true as a parameter to see each output

### Import the images

In [ ]:
import emoji
imageDict = dict()
fin = open('emoji_names.txt',"r")
lines = fin.readlines()
for i, el in enumerate(lines):
    imageDict[i+1] = emoji.emojize(el.strip())

In [ ]:
from random import choice
from random import sample
import sys

DobbleCard class below is passed a parameter, validcards, which we have previously generated based on the formula provided. It holds a getCard method which generates one random card from this list (originally, it was a dictionary with sets, but I prefered operating on lists, so changed the program to use lists instead).

In [ ]:
class DobbleCard():
    
    def __init__(self, validcards):
        self.validcards = validcards # initialise the list of validcards to pull from
    
    def getCard(self): # Get a random card from the valid card deck
        self.card = choice(self.validcards)
        return self.card
    
    #def testDeck(self):
    #    return self.validcards  ## Used only during testing

Our DobbleDeck class initialises with an empty deck. This will hold the selection of 8 random cards from our validcards in the class above, which will be used for the game.

The class has an add_cards method, which removes the requirement for us to call getCard 8 times individually within our deck (as in with our previous project, where we called Column() object 3 times for our slot).

On the first iteration of our game, play_card method will populat an empty list for the round so long as the list has less than 2 cards. These cards are required to be unique, which our if/while conditional statements provide. On each iteration of the next round (post 1st round), the play_card method checks whether the deck already has 1 card. If it does, then it will just populate the list with 1 additional card (to ensure that each round has 1 card from the previous round -- as per the rules of the game).

The remove card method removes 1 card from both the deck and the round cards list at the end of each round, to ensure this card is not played again.

In [ ]:
class DobbleDeck():
    def __init__(self):
        self.deck = [] # Initialise empty deck for number of cards to be played in the game.
    
    def add_cards(self,numOfCards): #
        """ This method adds the number of cards to the deck dependant on user input. 
        """
        counter = 0
        self.numOfCards = numOfCards
        while counter < numOfCards:
            x = DobbleCard(validcards).getCard()
            if not x in self.deck:
                self.deck.append(x) 
                counter += 1

    def play_card(self):
        """ This method gets 2 cards if roundCards is empty, or gets 1 card with each subsequent round, to ensure we are 
            always using 1 previous card.
        """
        self.roundCards = []
        if len(self.roundCards) == 0:
            while len(self.roundCards) < 2:
                card = choice(self.deck)
                if not card in self.roundCards:
                    self.roundCards.append(card)
        elif len(self.roundCards) == 1:
            while len(self.roundCards) < 2:
                card =  choice(self.deck)
                if not card in self.roundCards:
                    self.roundCards.append(card)
        return self.roundCards

    def remove_card(self): 
        """ This method removes the index 0 card from roundCards and also removes that card from the game deck. This also
            ensures that card[1] always moves to [0] so we are never stuck with the same card for more than
            1 round.
        """
        if len(self.deck) > 0:
            self.roundCards.remove(self.roundCards[0])
            self.deck.remove(self.roundCards[0])
            return self.roundCards
                
    #def viewDeck(self):        
    #    return self.deck  # used during testing phase

The playDobble function operates by initialising a DobbleDeck object, then initialising various counters for scoring and round counting.

In [ ]:
def playDobble():
    """ This function runs the entire dobble game by implementing the classes and methods built above. It does not take any 
        parameters since the object is initialised within the fuction and discarded at the end, as there is no reason to store
        any data from game to game.
    """
    p = DobbleDeck()
    scoreA = 0
    scoreB = 0
    roundsDrawn = 0
    roundCounter = 0
    cards = 0
    
    # ensure request is inbounds.
    
    while not cards in range(2,57):
        print("__________________________________________")
        print()
        print("Ensure you enter a value between 2 and 56.")
        print("__________________________________________")
        print()
        while True:
            cards =  input("Enter the amount of cards you'd like to play (<56): ")
            try:
                cards = int(cards)
            except ValueError:
                print("Sorry, you must enter an integer. Please try again.")
                continue
            else:
                break
    
    # we add one to ensure we play 2 rounds if user enters two cards (as in assignment documentation if user requests 
    # 5 cards, there are 5 rounds.) Required since each round is supposed to use 1 card from the previous round. Therefore,
    # without this, our last round would be missing a card.
     
    p.add_cards((cards+1)) 
    
    while roundCounter < cards:
        
        p.play_card()
        
        card1 = p.roundCards[0]
        card2 = p.roundCards[1]

        imgCard1= ""
        imgCard2= ""
    
    # build the emoji string based on number value of cards
    
        for i in card1:
            imgCard1 += imageDict[i]
        for i in card2:
            imgCard2 += imageDict[i]
        
    # print the emojis inline (card1 : card2)
        print(imgCard1[0] + " " + imgCard1[1] + " " + imgCard1[2] + "\t" + imgCard2[0] + " " + imgCard2[1] + " " + imgCard2[2])
        print(imgCard1[3] + " " + imgCard1[4] + " " + imgCard1[5] + "\t" + imgCard2[3] + " " + imgCard2[4] + " " + imgCard2[5])
        
        print(imgCard1[6] + " " + imgCard1[7] + "\t\t" + imgCard2[6] + " " + imgCard2[7])
        print()
        p.remove_card() # remove the card from the round and the deck
        
        winner = input("Who won, A or B, or a draw (enter D for draw): ")
        winnerLower = winner.lower()
        
        # check the winner and increment the score
        
        while winnerLower != "a" and winnerLower != "b" and winnerLower != "d":
            print("Sorry, I didn't understand who the winner was. A, B or D?")
            winner = input("Who won, A or B, or a draw (enter D for draw): ")
            winnerLower = winner.lower()
            
        if winnerLower == "a":
            scoreA += 1
        elif winnerLower == "b":
            scoreB += 1
        elif winnerLower == "d":
            roundsDrawn += 1
            
        roundCounter +=1
        
        # when the roundCounte equals the number of cards requested, analyse the results and ask if they want to play again.
        
        if roundCounter == cards:
            if scoreA > scoreB and scoreA > roundsDrawn:
                print()
                print("Player A is the winner with",scoreA,"points, versus player B with", scoreB,"points.")
            elif scoreB > scoreA and scoreB > roundsDrawn:
                print()
                print("Player B is the winner with",scoreB,"points, versus player A with", scoreA,"points.")
            else:
                print()
                print("Looks like it was a draw! Player A has",scoreA, "points and Player B has",scoreB,"points.")
        
            print("Would you like to play again?")
            yesNo = input("Type Y to play again, or anything else to quit: ")
            yesNoLower = yesNo.lower()
            
            # reset the counters if the users chose to contiun

            if yesNoLower == "y":
                scoreA = 0
                scoreB = 0
                roundsDrawn = 0
                roundCounter = 0
                cards = 0
            else:
                sys.exit() # exit if users don't want to continue playing

            #  ask for cards again if they chose to continue
            while not cards in range(2,57):
                print("__________________________________________")
                print()
                print("Ensure you enter a value between 2 and 56.")
                print("__________________________________________")
                print()
                while True:
                    cards =  input("Enter the amount of cards you'd like to play (<56): ")
                    try:
                        cards = int(cards)
                    except ValueError:
                        print("Sorry, you must enter an integer. Please try again.")
                        continue
                    else:
                        break

            p.add_cards((cards+1))

In [ ]:
playDobble()